In [1]:
import pandas as pd
import numpy as np

import requests
import praw

## Scrapping Data From Reddit

## Setting up a function `get_reddit()` that will scrape reddit posts and append content to a list that will be converted to a dataframe

#### Setting up `before` variable ensures we get unique posts each request

In [2]:
#Before is a pushshift parameter that scrapes the data before X

before = None

#### Setting up a list `reddit_list` that'll catch the data frames that come out of each interation of `get_reddit()` to later be concatinated back into a dataframe

In [3]:
reddit_list = []

#### Declaring a function `get_reddit()` that scrapes posts from reddit and appends to a `reddit_list`

In [4]:
def get_reddit(subreddit):   #subreddit is a parameter that
    
    global before  
    global df_test
    
    params = {
    "subreddit" : subreddit,
    "size" : 1000,      #Can only scrape 1,000 at a time
    "before": before    #before variable from above
    }
    
            #Getting the data from reddit and turning it into a DataFrame
    
    base_url = 'https://api.pushshift.io/reddit/submission/search'
    res = requests.get(base_url, params)
    reddit_df = pd.DataFrame(res.json()['data'])
    
            #Returning a Smaller Version of DataFrame
    
    reddit_df = reddit_df[['created_utc', 'title', 'selftext']]
    reddit_df['subreddit'] = subreddit
    
            #Overwriting the global veriable `before` so that the next instance gets different data
    
    before = reddit_df['created_utc'][-1:]
    
    reddit_list.append(reddit_df)

#### Resetting `before` variable to none to make sure the `get_reddit()` function grabs the most recent posts

In [5]:
before = None

#### <span style ='color:green'> Since Reddit only allows 1,000 pulls per request, we're grabbing 1,000 posts 10 times - we can make 200 requests per minute. 

#### Grabbing 10,000 posts from "AskMen" subreddit 

In [6]:
for i in range(10):
    get_reddit("AskMen")
    print(f"Interation {i+1}")

Interation 1
Interation 2
Interation 3
Interation 4
Interation 5
Interation 6
Interation 7
Interation 8
Interation 9
Interation 10


#### Resetting `before` variable to none to ensure the `get_reddit()` function grabs the most recent posts

In [7]:
before = None

#### Grabbing 10,000 posts from "AskWomen" subreddit

In [8]:
for i in range(10):
    get_reddit("AskWomen")
    print(f"Interation {i+1}")

Interation 1
Interation 2
Interation 3
Interation 4
Interation 5
Interation 6
Interation 7
Interation 8
Interation 9
Interation 10


#### Converting all the posts we grabbed into a dataframe

In [9]:
df = pd.concat(reddit_list)
df.head()

,created_utc,title,selftext,subreddit
0,1580515632,How do you muster up the courage to ask a girl...,[removed],AskMen
1,1580515616,Do men feel it if a condom accidentally falls ...,[removed],AskMen
2,1580515485,"Men, do you feel it if a condom accidentally f...",[removed],AskMen
3,1580515429,A girl you are dating says she had anxiety abo...,,AskMen
4,1580515379,"Men whose advice I desperately need, how often...",Help PLSSSSS,AskMen


#### Exporting as a CSV

In [11]:
df.to_csv("./reddit_scrape_df.csv", index = False)